In [ ]:
# Import your model
from hbv_model import  hbv_model

from response import response_routine_two_tanks
import numpy as np
import matplotlib.pyplot as plt

# --- Generate fake forcing data ---
np.random.seed(42)  # For reproducibility

n_days = 365

# Fake daily data
precipitation = np.random.gamma(shape=2, scale=2, size=n_days)  # random rainfall (mm)
temperature = np.random.normal(loc=5, scale=10, size=n_days)    # average temp (°C)
potential_et = np.clip(np.random.normal(loc=2, scale=0.5, size=n_days), 0, None)  # ET (mm)

# --- Define parameters ---
params = {
    'snow': {
        'TT': 0.0,
        'CFMAX': 3.0,
        'CFR': 0.05,
        'CWH': 0.1,
        'SFCF': 1.0
    },
    'soil': {
        'FC': 150.0,
        'BETA': 2.0,
        'LP': 0.7
    },
    'response': {
        'K0': 0.1,
        'K1': 0.05,
        'K2': 0.01,
        'UZL': 30.0,
        'PERC': 1.0
    }
}

# --- Initial conditions ---
initial_conditions = {
    'snowpack': 0.0,
    'liquid_water': 0.0,
    'soil_moisture': 50.0,
    'upper_storage': 5.0,
    'lower_storage': 20.0
}

# --- Arrays to store results ---
states = {key: [] for key in initial_conditions.keys()}
fluxes = {'discharge': [], 'quick_flow': [], 'intermediate_flow': [], 'baseflow': [], 
          'actual_et': [], 'recharge': [], 'runoff_soil': [], 'runoff_from_snow': []}

# --- Run the simulation ---
current_states = initial_conditions.copy()

for day in range(n_days):
    new_states, day_fluxes = hbv_model(
        precipitation[day], 
        temperature[day], 
        potential_et[day], 
        params, 
        current_states
    )
    
    # Save states
    for key in states:
        states[key].append(new_states[key])
    
    # Save fluxes
    for key in fluxes:
        fluxes[key].append(day_fluxes[key])
    
    # Update states
    current_states = new_states

# --- Convert to numpy arrays for easier plotting ---
for key in states:
    states[key] = np.array(states[key])

for key in fluxes:
    fluxes[key] = np.array(fluxes[key])

# --- Plot the results ---

# 1. Plot States
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(states['snowpack'], label='Snowpack (mm)')
plt.plot(states['soil_moisture'], label='Soil Moisture (mm)')
plt.title('Storage States')
plt.ylabel('Storage (mm)')
plt.legend()
plt.grid()

plt.subplot(3, 1, 2)
plt.plot(states['upper_storage'], label='Upper Storage (mm)')
plt.plot(states['lower_storage'], label='Lower Storage (mm)')
plt.ylabel('Storage (mm)')
plt.legend()
plt.grid()

plt.subplot(3, 1, 3)
plt.plot(precipitation, label='Precipitation (mm)', alpha=0.5)
plt.plot(temperature, label='Temperature (°C)', alpha=0.5)
plt.plot(potential_et, label='Potential ET (mm)', alpha=0.5)
plt.title('Forcings')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# 2. Plot Fluxes
plt.figure(figsize=(14, 8))

plt.plot(fluxes['discharge'], label='Total Discharge (mm/day)')
plt.plot(fluxes['quick_flow'], label='Quick Flow (mm/day)')
plt.plot(fluxes['intermediate_flow'], label='Intermediate Flow (mm/day)')
plt.plot(fluxes['baseflow'], label='Baseflow (mm/day)')
plt.title('Runoff Components')
plt.ylabel('Flow (mm/day)')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


NameError: name 'soil_routine' is not defined